# Vehicle Detection from images, videos, and Real-time detection with our trained model of custom dataSet with TensorFlow 2 Detection Model Zoo.

In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import pandas as pd
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# 1. Load Train Model From Checkpoint

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobilenet_v2_fpnlite_320x320' 
LABEL_MAP_NAME = 'label_map.pbtxt'
CHECKPOINT_NAME = 'ckpt-51'


In [3]:
paths = {
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'],CHECKPOINT_NAME )).expect_partial()

# Set the minimum detection score threshold
min_score_thresh = 0.6
def detect_fn(image):
    '''
    takes an image in tensor type and return detections
    '''
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detection Function

In [11]:
def vehicle_detection_data_frame(image, detect_fn):
    # convert image to tf tensor 
    input_tensor = tf.convert_to_tensor(image[np.newaxis, ...], dtype=tf.float32)
    
    # apply detect_fn on the input tensor
    detections = detect_fn(input_tensor)
    
    # extract required information from detections
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    # convert class ids to int
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    # filter detections with scores above threshold and convert bounding boxes to integer values
    scores = detections['detection_scores']
    boxes = detections['detection_boxes']
    classes = detections['detection_classes']
    num_detections = detections['num_detections']

    my_detections = []
    for i in range(num_detections):
        if scores[i] >= min_score_thresh:
            bbox = [int(a * b) for a, b in zip(boxes[i], image.shape[:2] * 2)]
            detection = {
                'score': scores[i],
                'class': classes[i],
                'box': bbox,
            }
            y1, x1, y2, x2 = bbox
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2) # 
            my_detections.append(detection)
    
    # return dataframe with detections or empty dataframe
    print(my_detections)
    if my_detections:
        df = pd.DataFrame(my_detections)
    else:
        df = pd.DataFrame(columns=['score', 'class', 'box'])
    
    return df

## 2. Detect from an Image

In [12]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'b1d9e136-9ab25cb3.jpg')

In [13]:
img = cv2.imread(IMAGE_PATH)
# img  = cv2.resize(img,(900,600))

df = vehicle_detection_data_frame(image=img,detect_fn=detect_fn)

# display image with bounding boxes
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


[{'score': 0.95691293, 'class': 0, 'box': [274, 470, 511, 751]}, {'score': 0.8801923, 'class': 0, 'box': [292, 0, 520, 322]}, {'score': 0.7778156, 'class': 0, 'box': [297, 361, 390, 475]}, {'score': 0.7642996, 'class': 0, 'box': [227, 876, 656, 1280]}]


## 3. Detect from a Video

In [14]:
cap = cv2.VideoCapture('video11.mp4')
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    cv2.resize(frame,(800,600))
    df = vehicle_detection_data_frame(image=frame,detect_fn=detect_fn)
    cv2.imshow('object detection',  cv2.resize(frame, (800, 600)))
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

[{'score': 0.8725246, 'class': 0, 'box': [659, 817, 920, 1126]}, {'score': 0.67323154, 'class': 0, 'box': [673, 1702, 1063, 1914]}]
[{'score': 0.80368334, 'class': 0, 'box': [653, 816, 921, 1129]}, {'score': 0.7018361, 'class': 0, 'box': [675, 1693, 1065, 1913]}]
[{'score': 0.84197813, 'class': 0, 'box': [653, 812, 923, 1123]}, {'score': 0.67077726, 'class': 0, 'box': [671, 1689, 1063, 1912]}]
[{'score': 0.82004005, 'class': 0, 'box': [649, 811, 921, 1123]}, {'score': 0.63916874, 'class': 0, 'box': [637, 1680, 1071, 1916]}]
[{'score': 0.83923256, 'class': 0, 'box': [656, 807, 923, 1130]}, {'score': 0.62725526, 'class': 0, 'box': [635, 1675, 1068, 1916]}]
[{'score': 0.8326284, 'class': 0, 'box': [657, 811, 923, 1126]}, {'score': 0.65444887, 'class': 0, 'box': [613, 1673, 1068, 1916]}]
[{'score': 0.7850558, 'class': 0, 'box': [646, 806, 916, 1123]}, {'score': 0.6532972, 'class': 0, 'box': [605, 1659, 1063, 1918]}]
[{'score': 0.81275386, 'class': 0, 'box': [644, 806, 918, 1118]}, {'score'